In [1]:
import os
import pickle
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from unsloth import is_bfloat16_supported


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
config  = {
    "model":{
        "name":"unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
        "max_seq_len":2048,
        "dtype":None,
        "load_in_4bit":True,
        "peft":{
            "r":32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
            "target_modules":["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
            "lora_alpha":16,
            "lora_dropout":0, # Supports any, but = 0 is optimized
            "bias":"none",    # Supports any, but = "none" is optimized
            # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
            "use_gradient_checkpointing":"unsloth", # True or "unsloth" for very long context
            "random_state":3407,
            "use_rslora":False,  # We support rank stabilized LoRA
            "loftq_config":None, # And LoftQ
        },
        
        "training_args":{
            "per_device_train_batch_size":64,
            "gradient_accumulation_steps":4,
            "warmup_steps":5,
            "num_train_epochs":1, # Set this for 1 full training run.
            "learning_rate":1e-4,
            "fp16":not is_bfloat16_supported(),
            "bf16":is_bfloat16_supported(),
            "logging_steps":1,
            "optim":"adamw_8bit",
            "weight_decay":0.01,
            "lr_scheduler_type":"linear",
            "output_dir":"outputs",
        },
        
    },
    "model_save":"saved_model",
    "seed":49,
}

In [3]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = config["model"]["name"],  # or choose "unsloth/Llama-3.2-1B"
    max_seq_length = config["model"]["max_seq_len"], # Choose any! We auto support RoPE Scaling internally!
    dtype = config["model"]["dtype"],# None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = config["model"]["load_in_4bit"], # Use 4bit quantization to reduce memory usage. Can be False.
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.43.4.
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.679 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = config["model"]["peft"]["r"],
    target_modules = config["model"]["peft"]["target_modules"],
    lora_alpha = config["model"]["peft"]["lora_alpha"],
    lora_dropout = config["model"]["peft"]["lora_dropout"],
    bias = config["model"]["peft"]["bias"],
    use_gradient_checkpointing = config["model"]["peft"]["use_gradient_checkpointing"],
    random_state = config["model"]["peft"]["random_state"],
    use_rslora = config["model"]["peft"]["use_rslora"],
    loftq_config = config["model"]["peft"]["loftq_config"],
)

Unsloth 2024.9.post4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
The datasets used are processed and contained in the datae=sets folder and please note EOS token is not added.


In [5]:
dataset_paths = [
    "datasets/ADME_BBB_Martins/processed",
    "datasets/PeptideMHC_MHC1_IEDB-IMGT_Nielsen/processed"
]

In [6]:
def open_pkl(loc):
    with open(loc, "rb") as f0: 
        data = pickle.load(f0)
        
    return data
        

def load_train_data(path):
    EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
    
    ds_path = os.path.join(path, "train.pkl")
    ds = open_pkl(ds_path)
    ds["text"] = list(map(lambda x: x+EOS_TOKEN,ds["text"]))
    ds = pd.DataFrame.from_dict(ds)
    return ds

def load_test_data(path, _set):
    EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
    
    ds_path = os.path.join(path, f"{_set}.pkl")
    ds = open_pkl(ds_path)
    ds["text"] = list(map(lambda x: x+EOS_TOKEN,ds["text"]))
    ds = pd.DataFrame.from_dict(ds)
    return ds
    

def load_and_prepdata(dataset_paths, ds_set="train", json_folder="final_data"):
    os.makedirs(json_folder,exist_ok=True)
    
    final_df = pd.DataFrame()
    if ds_set == "train":
        for i,p in enumerate(dataset_paths):
            dataset_i = load_train_data(p)
            final_df = pd.concat([final_df, dataset_i], axis=0, ignore_index=True)
  
    else:
        #check if change is required
        for i,p in enumerate(dataset_paths):
            dataset_i = load_test_data(p,ds_set)
            final_df = pd.concat([final_df, dataset_i], axis=0, ignore_index=True)
    
    final_df.to_json(
            os.path.join(json_folder, f"{ds_set}.json"),
            orient="records",
            lines=True
        )

In [7]:
load_and_prepdata(dataset_paths)
load_and_prepdata(dataset_paths, "valid")
load_and_prepdata(dataset_paths, "test")

In [8]:
dataset = load_dataset(
    "json",
    data_files={
        "train":"final_data/train.json",
        "valid":"final_data/valid.json",
        "test":"final_data/test.json",
    }
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating valid split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
model.config

LlamaConfig {
  "_name_or_path": "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": [
    128001,
    128008,
    128009
  ],
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [10]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import TrainingArguments

response_template = "### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset= dataset["valid"],
    dataset_text_field = "text",
    max_seq_length = config["model"]["max_seq_len"],
    dataset_num_proc = 4,
    data_collator=collator,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = config["model"]["training_args"]["per_device_train_batch_size"],
        gradient_accumulation_steps = config["model"]["training_args"]["gradient_accumulation_steps"],
        warmup_steps = config["model"]["training_args"]["warmup_steps"],
        num_train_epochs = config["model"]["training_args"]["num_train_epochs"], # Set this for 1 full training run.
        learning_rate = config["model"]["training_args"]["learning_rate"],
        fp16 = config["model"]["training_args"]["fp16"],
        bf16 = config["model"]["training_args"]["bf16"],
        logging_steps = config["model"]["training_args"]["logging_steps"],
        optim = config["model"]["training_args"]["optim"],
        weight_decay = config["model"]["training_args"]["weight_decay"],
        lr_scheduler_type = config["model"]["training_args"]["lr_scheduler_type"],
        seed = config["seed"],
        output_dir = config["model"]["training_args"]["output_dir"],
    ),
)

Map (num_proc=4):   0%|          | 0/131611 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/18801 [00:00<?, ? examples/s]

In [11]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX A5000. Max memory = 23.679 GB.
6.152 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 131,611 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 64 | Gradient Accumulation steps = 4
\        /    Total batch size = 256 | Total steps = 514
 "-____-"     Number of trainable parameters = 83,886,080


Step,Training Loss
1,3.124300
2,3.106300
3,3.097000


KeyboardInterrupt: 

In [13]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

10162.5699 seconds used for training.
169.38 minutes used for training.
Peak reserved memory = 19.844 GB.
Peak reserved memory for training = 17.119 GB.
Peak reserved memory % of max memory = 83.804 %.
Peak reserved memory for training % of max memory = 72.296 %.


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [10]:
model.save_pretrained(config["model_save"]) # Local saving
tokenizer.save_pretrained(config["model_save"])
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')